In [280]:
######### for this program to give accurate results, combined_df has to be run again if any code/name changes are there or
########## including any additions
########## companymaster has to be uptodate
######### shortlisted stocks - averages for nse/bse has to be updated
######## market cap from screener has to be updated

In [281]:
# importing the required modules

import os
import csv
import pandas as pd
from pathlib import Path
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, Paragraph, PageBreak
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, TA_LEFT

from platform import python_version
python_version()

'3.7.6'

In [282]:
### this program gives top 50 deliveries by value and volume for fno and non-fno. 
#### it also creates fno_csv and non_fno_csv which is used by multiple scrips program

### caution - if data file is already read, then it skips reading it. So if data is updated, close the program and restart it
#### to enable data file to be read again


In [283]:
base_folder = Path.cwd()
base_folder

WindowsPath('C:/Users/User/Dropbox/SQL Stock Market Data/Python Code from ChatGPT')

In [284]:
# list of file paths

master_file_path = Path.joinpath(base_folder, "Master File", "CompanyMaster.csv")
directory = Path.joinpath(Path(base_folder).parents[0], "Data Files")
output_path = Path.joinpath(base_folder, "Output File")
combined_file_path = Path.joinpath(Path(base_folder).parents[0], "Delivery Price Combined Files")
reports_path = Path.joinpath(base_folder,"PDF Reports")

column_order = ['NSE_Code','BSE_Code','Date','NSE_Delivery','BSE_Delivery']
print(reports_path)

C:\Users\User\Dropbox\SQL Stock Market Data\Python Code from ChatGPT\PDF Reports


In [285]:
file_names = ['BSE Gross Deliverable.txt', 'NSE Delivery.csv', 'NSE EQ.csv', 'EQ ISIN.csv']

while True:
    user_input = input("Enter a date in the 'yymmdd' format (or press Enter to exit): ")

    # Check if the user pressed Enter with an empty input
    if not user_input:
        print("Exiting the program.")
        break

    # Check if the input has 6 characters and consists of digits
    if len(user_input) == 6 and user_input.isdigit():
        all_files_exist = True
        updated_file_names = []

        for file_name in file_names:
            name, extension = file_name.split(".")
            print(name, extension)
            full_file_name = os.path.join(directory, f"{name} {user_input}.{extension}")
            file_name_date = f"{name} {user_input}.{extension}"
            print(f"full file name : {full_file_name} and file_name_date : {file_name_date}")
            if not os.path.exists(full_file_name):
                print()
                all_files_exist = False
                break
            updated_file_names.append(file_name_date)

        if all_files_exist:
            print(f"All required files for the date '{user_input}' exist.")
            # You can use the updated_file_names list for further processing
            print("Updated file names:", updated_file_names)
            break  # Exit the loop if all files exist
        else:
            print("Not all required files exist. Please check the files.")
    else:
        print("Invalid input. Please enter a date in the 'yymmdd' format (e.g., 220101).")
        

Enter a date in the 'yymmdd' format (or press Enter to exit): 231222
BSE Gross Deliverable txt
full file name : C:\Users\User\Dropbox\SQL Stock Market Data\Data Files\BSE Gross Deliverable 231222.txt and file_name_date : BSE Gross Deliverable 231222.txt
NSE Delivery csv
full file name : C:\Users\User\Dropbox\SQL Stock Market Data\Data Files\NSE Delivery 231222.csv and file_name_date : NSE Delivery 231222.csv
NSE EQ csv
full file name : C:\Users\User\Dropbox\SQL Stock Market Data\Data Files\NSE EQ 231222.csv and file_name_date : NSE EQ 231222.csv
EQ ISIN csv
full file name : C:\Users\User\Dropbox\SQL Stock Market Data\Data Files\EQ ISIN 231222.csv and file_name_date : EQ ISIN 231222.csv
All required files for the date '231222' exist.
Updated file names: ['BSE Gross Deliverable 231222.txt', 'NSE Delivery 231222.csv', 'NSE EQ 231222.csv', 'EQ ISIN 231222.csv']


In [286]:
nse_dely_file = [updated_file_names[1]]
nse_price_file = [updated_file_names[2]]
bse_dely_file = [updated_file_names[0]]
bse_price_file = [updated_file_names[3]]
print(nse_dely_file,nse_price_file,bse_dely_file,bse_price_file)

['NSE Delivery 231222.csv'] ['NSE EQ 231222.csv'] ['BSE Gross Deliverable 231222.txt'] ['EQ ISIN 231222.csv']


In [287]:
def nse_data(nse_file_name):
    nse_df = pd.DataFrame(columns=['NSE_Code','Date','NSE_Volume','NSE_Delivery','Segment'])
    dely_segment = ['EQ','BE','SM','ST']
    
    for file_name in nse_file_name:
        file_path = os.path.join(directory, file_name)
        date = file_name[13:19]
        
        with open(file_path,'r') as file:
            reader = csv.reader(file)
            for i, line in enumerate(reader, start=1):
                if i >= 5:
                    line_str = ','.join(line)
                    data = line_str.split(',')
                    if data[3] in dely_segment:
                        nse_code = data[2]
                        dely = data[5]
                        segment = data[3]
                        volume = data[4]
                        df_to_append = pd.DataFrame({'NSE_Code': [nse_code], 'Date': [date], 'NSE_Volume':volume,
                                                     'NSE_Delivery': [dely], 'Segment': [segment]})
                        nse_df = pd.concat([nse_df, df_to_append], ignore_index=True)
    return nse_df
                
 
def bse_data(bse_file_name):
    # Create an empty DataFrame to store the results
    bse_df = pd.DataFrame(columns=['BSE_Code', 'Date','BSE_Delivery'])
    dely_segment = ['EQ','BE','SM','ST']

    for file_name in bse_file_name:
        file_path = os.path.join(directory, file_name)
        date = file_name[22:28] 
    
        with open(file_path, 'r') as file:
            reader = csv.reader(file)
            for i, line in enumerate(reader, start=1):
                if i >= 2:  # Start reading from line 5
                    line_str = ','.join(line)
                    data = line_str.split('|')
                    dely = data[2]
                    bse_code = data[1]
                    df_to_append = pd.DataFrame({'BSE_Code': [bse_code], 'Date': [date], 'BSE_Delivery':[dely]}) 
                    bse_df = pd.concat([bse_df,df_to_append], ignore_index=True)               
    return bse_df

In [288]:
def nse_price(nse_price_file):
    nse_df = pd.DataFrame(columns=['NSE_Code', 'Date', 'Price', 'PrevClose', 'Segment', 'Traded'])
    dely_segment = ['EQ', 'BE', 'SM', 'ST']

    for file_name in nse_price_file:
        file_path = os.path.join(directory, file_name)
        date = file_name[7:13]

        with open(file_path, 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                if row and row[1] in dely_segment:  # Check if the row is not empty and segment matches
                    nse_code = row[0]
                    price = row[5]
                    prev_close = row[7]
                    segment = row[1]
                    traded = row[8]
                    df_to_append = pd.DataFrame({'NSE_Code': [nse_code], 'Date': [date], 'Price': [price], 
                                                 'PrevClose': [prev_close],'Segment': [segment], 'Traded': [traded]})
                    nse_df = pd.concat([nse_df, df_to_append], ignore_index=True)
    return nse_df

def bse_price(bse_price_file):
    bse_df = pd.DataFrame(columns=['BSE_Code', 'Date', 'Price', 'PrevClose', 'BSE_Volume','Segment'])
    dely_segment = ['EQ', 'BE', 'SM', 'ST']

    for file_name in bse_price_file:
        file_path = os.path.join(directory, file_name)
        date = file_name[8:14]

        with open(file_path, 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                #if row and row[1] in dely_segment:  # Check if the row is not empty and segment matches
                    bse_code = row[0]
                    price = row[7]
                    prev_close = row[9]
                    bse_group = row[2]
                    volume = row[11]
                    df_to_append = pd.DataFrame(
                                {'BSE_Code': [bse_code], 
                                'Date': [date], 
                                'Price': [price], 
                                'PrevClose': [prev_close],
                                'BSE_Volume':volume ,
                                'Segment': [bse_group]}
                             )
                    bse_df = pd.concat([bse_df, df_to_append], ignore_index=True)

    # Drop the first row, which is a header row, and reset the index
    bse_df = bse_df.iloc[1:].reset_index(drop=True)
    return bse_df

In [289]:
def add_nse_prefix(df, column_name):
    """Create a copy of the DataFrame to avoid SettingWithCopyWarning"""
    df = df.copy()

    # Add "NSE:" prefix to the specified column
    df.loc[:, column_name] = "NSE:" + df[column_name].astype(str)
    return df

In [290]:
### getting unique symbols from fno
nse_fno_file = sorted([file for file in os.listdir(directory) if file.startswith('NSE FNO')],reverse=True)[:1]

for file_name in nse_fno_file:
    file_path = os.path.join(directory, file_name)
    fno_df = pd.read_csv(file_path)
    fno_df = fno_df.drop_duplicates(subset ='SYMBOL')
    fno_symbols = fno_df['SYMBOL']
    fno_symbols = fno_symbols.reset_index(drop=True)

In [291]:
print('processing nse_dely')
nse_dely_data = nse_data(nse_dely_file)

print('processing bse_dely')
bse_dely_data = bse_data(bse_dely_file)

print('processing nse_price')
nse_price_data = nse_price(nse_price_file)

print('processing bse_price')
bse_price_data = bse_price(bse_price_file)

print('all done, proceed')

processing nse_dely
processing bse_dely
processing nse_price
processing bse_price
all done, proceed


In [292]:
nse_dely_data['NSE_Delivery'] = nse_dely_data['NSE_Delivery'].astype(int)
nse_dely_data['NSE_Volume'] = nse_dely_data['NSE_Volume'].astype(int)
nse_price_data['Price'] = nse_price_data['Price'].astype(float)
nse_price_data['PrevClose'] = nse_price_data['PrevClose'].astype(float)
nse_price_data['Traded'] = nse_price_data['Traded'].astype(int)

bse_dely_data['BSE_Delivery'] = bse_dely_data['BSE_Delivery'].astype(int)
bse_price_data['Price'] = bse_price_data['Price'].astype(float)
bse_price_data['PrevClose'] = bse_price_data['PrevClose'].astype(float)
bse_price_data['BSE_Volume'] = bse_price_data['BSE_Volume'].astype(int)


In [293]:
# nse_dely_data.info()
# bse_dely_data.info()
# nse_price_data.info()
# bse_price_data.info()

In [294]:
### nse_dely_data has NSE_Code, Date, NSE_Delivery, Segment
### bse_dely_data has BSE_Code, Date, BSE_Delivery

### nse_price_data has NSE_Code	Date	Price	PrevClose	Segment	Traded
### bse_price_data has BSE_Code	Date	Price	PrevClose	Segment

In [295]:
## this section processes nse delivery and price file. Makes sure that stocks which are in BE segment and 
## therefore dont appear in nse delivery are taken from price file and new nse merged df is generated

### merging nse price and delivery files
nse_merged_df_out = pd.merge(nse_dely_data,nse_price_data, on = 'NSE_Code', how = 'outer')

### processing merged file
# Replace 'NSE_Delivery' with 'Traded' where 'NSE_Delivery' is NaN
nse_merged_df_out['NSE_Delivery'] = nse_merged_df_out['NSE_Delivery'].fillna(nse_merged_df_out['Traded'])

# Replace nse volume with nse delivery where nse volume is NaN
nse_merged_df_out['NSE_Volume'] = nse_merged_df_out['NSE_Volume'].fillna(nse_merged_df_out['NSE_Delivery'])

# Replace 'Date_x' with 'Date_y' where 'Date_x' is NaN
nse_merged_df_out['Date_x'] = nse_merged_df_out['Date_x'].fillna(nse_merged_df_out['Date_y'])

# Replace 'Segment_x' with 'Segment_y' where 'Segment_x' is NaN
nse_merged_df_out['Segment_x'] = nse_merged_df_out['Segment_x'].fillna(nse_merged_df_out['Segment_y'])

# Rename 'Segment_x' to 'Segment'
nse_merged_df_out.rename(columns={'Segment_x': 'Segment'}, inplace=True)

# Rename 'Date_x' to 'Date'
nse_merged_df_out.rename(columns={'Date_x': 'Date'}, inplace=True)

# Drop the 'Date_y' column
nse_merged_df_out.drop(columns=['Date_y'], inplace=True)

# Drop the 'Segment_y' column
nse_merged_df_out.drop(columns=['Segment_y'], inplace=True)

# Delete the 'Traded' column after replacement
nse_merged_df_out.drop(columns=['Traded'], inplace=True)

# Reset the index
nse_merged_df_out.reset_index(drop=True, inplace=True)

# add traded_value column

nse_merged_df_out['Traded_Value_lacs'] = (nse_merged_df_out['NSE_Delivery'] * nse_merged_df_out['Price'] / 100000).round(2)

#print(nse_merged_df_out)

nse_col_order = ['Date','NSE_Code','NSE_Volume','NSE_Delivery','Price','PrevClose','Traded_Value_lacs','Segment']
nse_merged_df_out = nse_merged_df_out[nse_col_order]

#### nse_merged_df_out is NSE_Code	Date	NSE_Delivery	Segment	Price	PrevClose	Traded_Value_lacs


In [296]:
nse_merged_df_out.head()

,Date,NSE_Code,NSE_Volume,NSE_Delivery,Price,PrevClose,Traded_Value_lacs,Segment
0,231222,20MICRONS,1118134.0,295886.0,189.45,170.50,560.56,EQ
1,231222,21STCENMGM,6944.0,6944.0,32.25,31.65,2.24,EQ
2,231222,360ONE,611023.0,186447.0,657.90,635.70,1226.63,EQ
3,231222,3IINFOLTD,1388245.0,503089.0,45.20,44.25,227.40,EQ
4,231222,3MINDIA,1812.0,671.0,31182.00,31358.30,209.23,EQ


In [297]:
###merging bse price and delivery files

bse_merged_df_out = pd.merge(bse_dely_data, bse_price_data, on = 'BSE_Code', how = 'outer')


### processing bse merged file

bse_merged_df_out['Traded_Value_lacs'] = (bse_merged_df_out['BSE_Delivery'] * bse_merged_df_out['Price'] / 100000).round(2)

# Rename 'Date_x' to 'Date'
bse_merged_df_out.rename(columns={'Date_x': 'Date'}, inplace=True)

# Drop the 'Date_y' column
bse_merged_df_out.drop(columns=['Date_y'], inplace=True)
bse_merged_df_out = bse_merged_df_out[bse_merged_df_out['BSE_Code'] <= '599999']

bse_merged_df_out.reset_index(drop=True, inplace = True)

#print(bse_merged_df_out)

bse_col_order = ['Date','BSE_Code','BSE_Volume','BSE_Delivery','Price','PrevClose','Traded_Value_lacs','Segment']
bse_merged_df_out = bse_merged_df_out[bse_col_order]

#### bse_merged_df_out is BSE_Code	Date	BSE_Delivery	Price	PrevClose	Segment	Traded_Value_lacs

In [298]:
### saving nse and bse files separately

nse_file = str(combined_file_path) +"\\"+ 'nse_' + user_input + '.csv'
bse_file = str(combined_file_path) + "\\"+'bse_' + user_input + '.csv'

#df.to_csv('sample.csv', index=False)

nse_merged_df_out.to_csv(nse_file, index = False)
bse_merged_df_out.to_csv(bse_file, index = False)


In [299]:
#nse_merged_df_out.info()
#nse_merged_df_out

In [300]:
#bse_merged_df_out.info()
#bse_merged_df_out

In [301]:
## reading master file
### master file (csv) is Company Name	CD_NSE Symbol	CD_BSE Code	CD_Is Subsidiary	CD_Sector	CD_Industry1

#### TO FIX - MASTER FILE DOES NOT HAVE SM SEGMENT CODES

master_file = pd.read_csv(master_file_path, dtype={'CD_BSE Code': str})


###df.rename(columns={'old_column_name': 'new_column_name'}, inplace=True)
##df.rename(columns={'old_column_name': 'new_column_name'}, inplace=True)
### Inplace method (modifies the original DataFrame)
###df.drop(columns=['column1', 'column2'], inplace=True)

master_file.drop(columns=['CD_Is Subsidiary','CD_Sector','CD_Industry1'], inplace = True )
master_file.rename(columns={'CD_NSE Symbol': 'NSE_Code', 'CD_BSE Code': 'BSE_Code'}, inplace= True )


#### master_file df is Company Name	NSE_Code	BSE_Code

#print(master_file)

In [302]:
## Merge NSE df with master file 

master_file = pd.merge(master_file,nse_merged_df_out, on= "NSE_Code", how = 'outer')
#master_file.dropna(subset=['Company Name'], inplace=True)


## merge bse df  with master file 
master_file = pd.merge(master_file,bse_merged_df_out, on ="BSE_Code", how = "outer")
#master_file.dropna(subset=['Company Name'], inplace=True)

# Drop records where both 'NSE_Code' and 'BSE_Code' are NaN or blank
master_file.dropna(subset=['NSE_Code', 'BSE_Code'], how='all', inplace=True)

# Reset the index after dropping rows
master_file.reset_index(drop=True, inplace=True)

# Drop records where both 'NSE_Delivery' and 'BSE_Delivery' are 0 or blank or NaN
master_file = master_file[~((master_file['NSE_Delivery'] == 0) & (master_file['BSE_Delivery'] == 0))]

# Reset the index after dropping rows
master_file.reset_index(drop=True, inplace=True)


# Trim "Company Name" column to a maximum length of 20 characters
master_file['Company Name'] = master_file['Company Name'].str.slice(0, 30)


# Reset the index after dropping rows
master_file.reset_index(drop=True, inplace=True)

#print(master_file)

In [303]:

# Identify the data types of each column
column_data_types = master_file.dtypes

# Replace NaN values in object columns with empty strings ("")
object_columns = master_file.select_dtypes(include='object').columns
master_file[object_columns] = master_file[object_columns].fillna("")

# Replace NaN values in float/integer columns with 0
numeric_columns = master_file.select_dtypes(include=['float64', 'int64']).columns
master_file[numeric_columns] = master_file[numeric_columns].fillna(0)

# Print the resulting DataFrame
#print(master_file)

#master_file.info()

In [304]:
# Define a custom function to replace Price_x with Price_y if Price_x is 0
def replace_price_x(row):
    if row['Price_x'] == 0:
        return row['Price_y']
    else:
        return row['Price_x']

    
def replace_prevclose_x(row):
    if row['PrevClose_x'] == 0:
        return row['PrevClose_y']
    else:
        return row['PrevClose_x']
    
    
def replace_segment(row):
    if row['Segment_x'] == "":
        return row['Segment_y']
    else:
        return row['Segment_x']
    
def replace_date(row):
    if row['Date_x'] == "":
        return row['Date_y']
    else:
        return row['Date_x']

# Apply the custom function to the DataFrame
master_file['Price_x'] = master_file.apply(replace_price_x, axis=1)
master_file['PrevClose_x'] = master_file.apply(replace_prevclose_x, axis=1)

# Rename 'Price_x' column to 'Price'
master_file.rename(columns={'Price_x': 'Price'}, inplace=True)
master_file.rename(columns={'PrevClose_x':'PrevClose'}, inplace=True)

### segment

master_file['Segment_x'] = master_file.apply(replace_segment, axis = 1)
master_file.rename(columns={'Segment_x' :'Segment'}, inplace = True)

#Date
master_file['Date_x'] = master_file.apply(replace_date, axis = 1)
master_file.rename(columns={'Date_x' :'Date'}, inplace = True)

### add total delivery column
master_file['Total_Delivery'] = master_file['NSE_Delivery'] + master_file['BSE_Delivery']
master_file['Total_Delivery'] = master_file['Total_Delivery'].astype(int)
master_file = master_file[master_file['Total_Delivery'] >= 1]

### add total volume column
master_file['Total_Volume'] = master_file['NSE_Volume'] + master_file['BSE_Volume']
master_file['Total_Volume'] = master_file['Total_Volume'].astype(int)


## add total traded value colum

master_file['Total_Traded_Value'] = ( master_file['Total_Delivery'] * master_file['Price'] ) / 100000



### reorder columns

columns_to_copy = ['Date','Company Name','Total_Volume','Total_Delivery', 'Price','PrevClose','Total_Traded_Value' ,'NSE_Code','BSE_Code','Segment']

# Create a new DataFrame 'final_master_file' with the desired columns
#final_master_file = master_file[columns_to_copy].copy()



In [305]:
#master_file

In [306]:

master_file_columns = ['Date','Company Name','NSE_Code','BSE_Code','Total_Volume','Total_Delivery', 'Price','PrevClose',
                       'Total_Traded_Value' , 'NSE_Volume','NSE_Delivery','BSE_Volume','BSE_Delivery','Segment']
final_master_file = master_file[master_file_columns]
master_file_csv = str(combined_file_path) + "\\"+'combined_' + user_input + '.csv'

#df.to_csv('sample.csv', index=False)

final_master_file.to_csv(master_file_csv, index = False)

In [307]:
#### create sme file
sme_segment = ['SM','ST','M ','MT']
sme_delivery = final_master_file[final_master_file['Segment'].isin(sme_segment)]
sme_delivery.loc[sme_delivery['NSE_Code'] == '', 'NSE_Code'] = sme_delivery['BSE_Code']
sme_delivery.reset_index(drop=True, inplace=True)

# Drop records where Total_Traded_Value is below 1000
final_master_file = final_master_file[final_master_file['Total_Traded_Value'] >= 1000]


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [308]:
###creating separate file for fno delivery and non fno delivery

# Filter final_master_file to get fno_delivery by checking if NSE_Code is in fno_symbols
fno_delivery = final_master_file[final_master_file['NSE_Code'].isin(fno_symbols)]
fno_delivery_final = fno_delivery[['NSE_Code', 'NSE_Volume','Total_Delivery', 'Price','PrevClose', 
                                   'Total_Traded_Value']].copy()


# Create non_fno_delivery by dropping rows where NSE_Code is in fno_symbols
non_fno_delivery = final_master_file[~final_master_file['NSE_Code'].isin(fno_symbols)]

# Reset index inplace for fno_delivery and non_fno_delivery
fno_delivery_final.reset_index(drop=True, inplace=True)
non_fno_delivery.reset_index(drop=True, inplace=True)

# Replace empty strings in NSE_Code with BSE_Code
#non_fno_delivery.loc[non_fno_delivery['NSE_Code'] == '', 'NSE_Code'] = non_fno_delivery['BSE_Code']


# Replace empty strings in NSE_Code with BSE_Code using a copy
non_fno_delivery_copy = non_fno_delivery.copy()
non_fno_delivery_copy.loc[non_fno_delivery_copy['NSE_Code'] == '', 'NSE_Code'] = non_fno_delivery_copy['BSE_Code']

In [309]:
# Replace empty strings in NSE_Code with BSE_Code
#non_fno_delivery['NSE_Code'] = non_fno_delivery.apply(lambda row: row['BSE_Code'] if row['NSE_Code'] == "" else row['NSE_Code'], axis=1)

#non_fno_delivery.loc[non_fno_delivery['NSE_Code'] == '', 'NSE_Code'] = non_fno_delivery['BSE_Code']


#sme_delivery
#non_fno_delivery

In [310]:
segment_var = ['SM','ST','M ']
#final_master_file.info()
 ##Count the records in each segment
segment_counts = master_file['Segment'].value_counts()

#print(segment_counts)

In [311]:
nse_dely_date = user_input
### creating top 50 by value and by volume for fno and non-fno

def top_n(df, n, column_name, trade_amt):
    df_filtered = df[df['Total_Traded_Value'] > trade_amt]
    top_n = df_filtered.nlargest(n, column_name).sort_values(column_name, ascending=False)
    
    # Format and right-align specific columns
    columns_to_format = ['Total_Delivery', 'Price', 'PrevClose', 'Total_Traded_Value']
    column_widths = {'Total_Delivery': 15, 'Price': 12, 'PrevClose': 12, 'Total_Traded_Value': 18}

    for col in columns_to_format:
        if col == 'Total_Delivery':
            top_n[col] = top_n[col].apply(lambda x: f"{int(x):,}".rjust(column_widths[col]))
        else:
            top_n[col] = top_n[col].apply(lambda x: f"{x:,.2f}".rjust(column_widths[col]))

    return top_n



df_value_50 = top_n(non_fno_delivery,50,'Total_Traded_Value', 250)
df_volume_50 = top_n(non_fno_delivery,50,'Total_Delivery', 250)

fno_value_50 = top_n(fno_delivery,50,'Total_Traded_Value', 1000)
fno_volume_50 = top_n(fno_delivery,50,'Total_Delivery', 1000)

fno_columns = ['NSE_Code', 'Total_Delivery','Price','PrevClose','Total_Traded_Value']

fno_value_50 = fno_value_50[fno_columns]
fno_volume_50 = fno_volume_50[fno_columns]

sme_value_50 = top_n(sme_delivery,50,'Total_Traded_Value', 50)

###########################

###########################

## write to excel -- non_fno value and volume is merged by concat and saved for multiple scrips report (7-day history)
#### sme_csv is similarly saved for 7-day history report

df_non_fno = pd.concat([df_value_50, df_volume_50])
df_non_fno_date = df_non_fno
df_non_fno_wl = df_non_fno[['NSE_Code']]

df_fno = pd.concat([fno_value_50, fno_volume_50])

# Reset the index of the series
df_non_fno_wl.reset_index(drop=True, inplace=True)

#add NSE: prefix to NSE_Code
df_non_fno_wl_nse = add_nse_prefix(df_non_fno_wl,'NSE_Code')


### define csv files
output_non_fno = str(output_path) +'\\'+"0_non_fno.csv"
output_non_fno_date = str(output_path) + "\\"+nse_dely_date + "_0_non_fno_.csv"
output_non_fno_wl = str(output_path) + "\\"+ '_' + nse_dely_date +"_0_non_fno_wl.csv"


output_fno = str(output_path) + "\\"+ "fno.csv"

output_sme_nodate = str(output_path)+ "\\" + 'sme_csv.csv'
output_sme = str(output_path)+ "\\" + nse_dely_date + '_0_sme_.csv'

### write to csv
df_non_fno.to_csv(output_non_fno, index = False)
df_non_fno.to_csv(output_non_fno_date, index = False)
df_non_fno_wl_nse.to_csv(output_non_fno_wl, index = False)

df_fno.to_csv(output_fno, index = False)

sme_delivery.to_csv(output_sme, index=False)
sme_value_50.to_csv(output_sme_nodate)

In [312]:
def print_formatted_rows(df, columns_to_select):
    # Iterate over the DataFrame and print each row
    for i, (_, row) in enumerate(df.iterrows(), 1):
        p_coname = row[columns_to_select[0]].ljust(40)
        p_dely = f"{row[columns_to_select[1]]:,.0f}".rjust(15)
        p_price = f"{row[columns_to_select[2]]:,.2f}".rjust(12)
        p_prevclose = f"{row[columns_to_select[3]]:,.2f}".rjust(12)
        p_trade_value = f"{row[columns_to_select[4]]:,.2f}".rjust(18)
        p_nse_code = row[columns_to_select[5]].ljust(10)
        
        print(f"{p_coname}{p_dely}{p_price}{p_prevclose}{p_trade_value}    {p_nse_code}")

        # Check if the current row index is a multiple of 5, and insert a blank line
        if i % 5 == 0:
            print()

def fno_formatted_rows(df, columns_to_select):
    # Iterate over the DataFrame and print each row
    for i, (_, row) in enumerate(df.iterrows(),1):
        p_nse_code = row[columns_to_select[0]].ljust(10)
        p_dely = f"{row[columns_to_select[1]]:,.0f}".rjust(15)
        p_price = f"{row[columns_to_select[2]]:,.2f}".rjust(12)
        p_prevclose = f"{row[columns_to_select[3]]:,.2f}".rjust(12)
        p_trade_value = f"{row[columns_to_select[4]]:,.2f}".rjust(12)
        
        print(f"{p_nse_code}{p_dely}{p_price}{p_prevclose}{p_trade_value}")
        
        # Check if the current row index is a multiple of 5, and insert a blank line
        if i % 5 == 0:
            print()

In [313]:
def fno_formatted_rows(df, columns_to_select):
    # Iterate over the DataFrame and print each row
    for i, (_, row) in enumerate(df.iterrows(), 1):
        p_coname = str(row[columns_to_select[0]]).ljust(40)

        # Handle 'Total_Delivery' column (index 1)
        total_delivery = row[columns_to_select[1]]
        if isinstance(total_delivery, (int, float)):
            p_dely = f"{total_delivery:,.0f}"
        elif isinstance(total_delivery, str) and total_delivery.replace(',', '').isdigit():
            p_dely = f"{float(total_delivery):,.0f}"
        else:
            p_dely = str(total_delivery)

        # Clean the 'Price' column before converting to float
        price_str = ''.join(char for char in row[columns_to_select[2]] if char.isdigit() or char == '.')
        p_price = f"{float(price_str.replace(',', '')):,.2f}".rjust(12)

        # Clean the 'PrevClose' column before converting to float
        prevclose_str = ''.join(char for char in row[columns_to_select[3]] if char.isdigit() or char == '.')
        p_prevclose = f"{float(prevclose_str.replace(',', '')):,.2f}".rjust(12)

        # Clean the 'Total_Traded_Value' column before converting to float
        trade_value_str = ''.join(char for char in row[columns_to_select[4]] if char.isdigit() or char == '.')
        p_trade_value = f"{float(trade_value_str.replace(',', '')):,.2f}".rjust(18)

        

        print(f"{p_coname}{p_dely}{p_price}{p_prevclose}{p_trade_value}    ")

        # Check if the current row index is a multiple of 5, and insert a blank line
        if i % 5 == 0:
            print()

def print_formatted_rows(df, columns_to_select):
    # Iterate over the DataFrame and print each row
    for i, (_, row) in enumerate(df.iterrows(), 1):
        p_coname = str(row[columns_to_select[0]]).ljust(40)
        
        # Handle 'Total_Delivery' column (index 1)
        total_delivery = row[columns_to_select[1]]
        if isinstance(total_delivery, (int, float)):
            p_dely = f"{total_delivery:,.0f}"
        elif isinstance(total_delivery, str) and total_delivery.replace(',', '').isdigit():
            p_dely = f"{float(total_delivery):,.0f}"
        else:
            p_dely = str(total_delivery)

        # Clean the 'Price' column before converting to float
        price_str = ''.join(char for char in row[columns_to_select[2]] if char.isdigit() or char == '.')
        p_price = f"{float(price_str.replace(',', '')):,.2f}".rjust(12)

        # Clean the 'PrevClose' column before converting to float
        prevclose_str = ''.join(char for char in row[columns_to_select[3]] if char.isdigit() or char == '.')
        p_prevclose = f"{float(prevclose_str.replace(',', '')):,.2f}".rjust(12)

        # Clean the 'Total_Traded_Value' column before converting to float
        trade_value_str = ''.join(char for char in row[columns_to_select[4]] if char.isdigit() or char == '.')
        p_trade_value = f"{float(trade_value_str.replace(',', '')):,.2f}".rjust(18)

        p_nse_code = str(row[columns_to_select[5]]).ljust(10)

        print(f"{p_coname}{p_dely}{p_price}{p_prevclose}{p_trade_value}    {p_nse_code}")

        # Check if the current row index is a multiple of 5, and insert a blank line
        if i % 5 == 0:
            print()

In [314]:
report_prefix_name = f"{user_input}_0_Top 50 deliveries_value_volume_non_fno_fno_sme.pdf"
pdf_name = os.path.join(reports_path,report_prefix_name)
print(pdf_name)

C:\Users\User\Dropbox\SQL Stock Market Data\Python Code from ChatGPT\PDF Reports\231222_0_Top 50 deliveries_value_volume_non_fno_fno_sme.pdf


In [315]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, PageBreak
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import Paragraph
from pathlib import Path
from reportlab.pdfbase.pdfmetrics import registerFontFamily

registerFontFamily('Vera', normal='Vera', bold='VeraBd', italic='VeraIt', boldItalic='VeraBI')

columns_to_select = ['Company Name', 'Total_Delivery', 'Price', 'PrevClose', 'Total_Traded_Value', 'NSE_Code']
fno_columns_to_select = ['NSE_Code', 'Total_Delivery', 'Price', 'PrevClose', 'Total_Traded_Value']
dfs = [df_value_50, df_volume_50, fno_value_50, fno_volume_50, sme_value_50]
report_titles = [
    f"Top 50 delivery by value NON-FNO {user_input}",
    f"Top 50 delivery by volume NON-FNO {user_input}",
    f"Top 50 FNO Delivery by value {user_input}",
    f"Top 50 FNO Delivery by volume {user_input}",
    f"Top 50 SME delivery by value {user_input}"
]

#report_prefix_name = f"{user_input}_0_Top 50 deliveries by value volume in non_fno fno and sme.pdf"
#report_path = Path.joinpath(Path(base_folder).parents[0], "Reports", report_prefix_name)
report_prefix_name = f"{user_input}_0_Top 50 deliveries_value_volume_non_fno_fno_sme.pdf"
report_path = os.path.join(reports_path,report_prefix_name)

# Define the style for column headers without grid lines
column_header_style = [
    ('BACKGROUND', (0, 0), (-1, 0), colors.lightgrey),
    ('TEXTCOLOR', (0, 0), (-1, 0), colors.black),
    ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
]


def create_table_with_header(df, df_name, add_column_names=False, table_width=None):
    """create a table with header and data"""
    table_data = [df.columns.tolist()] + df.values.tolist()

    # Add empty rows for spacing after every 5 records
    for i in range(6, len(table_data), 6):
        table_data.insert(i, [''] * len(df.columns))
        
    styles = getSampleStyleSheet()
    h2_style = styles['Normal']
    h2_style.fontSize = 9
    h2_style.fontName = 'Calibri'

    if table_width is None:
        t = Table(table_data, repeatRows=1, style=h2_style)
    else:
        colwdth = [170, 80, 67, 68, 105]
        t = Table(table_data, repeatRows=1, colWidths=colwdth)  # Adjust colWidths for left alignment

    style = [
        ('ALIGN', (0, 0), (0, -1), 'LEFT'),  # Left align first column
        ('ALIGN', (-1, 0), (-1, -1), 'LEFT'),  # Left align last column
        ('ALIGN', (1, 0), (-2, -1), 'RIGHT'),  # Right align other columns
    ]

    # repeatRows=1 to add the first column to all the pages
    t.setStyle(column_header_style)

    if "FNO" in str(df_name) and "NON-FNO" not in str(df_name):
        style = [
            ('ALIGN', (-1, 0), (-1, -1), 'LEFT'),  # Left align first column
            ('ALIGN', (1, 0), (-1, -1), 'RIGHT'),  # Right align other columns (including last)
        ]

    # Set title row color for the first page
    if getattr(create_table_with_header, 'first_page', True):
        style.extend([
            ('BACKGROUND', (0, 0), (-1, 0), colors.lightgrey),  # Background color for the title row
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.black),
        ])
        create_table_with_header.first_page = False

    t.setStyle(style)

    # Remove table borders
    t.setStyle([('LINEBELOW', (0, 0), (-1, 0), 0, colors.white),  # Hide line below header
                ('GRID', (0, 0), (-1, -1), 0, colors.white)])  # Hide grid lines

    return t


def add_headers_and_footers(canvas, doc):
    """Add header and footer here"""
    page_num = canvas.getPageNumber()
    text = f"Page {page_num}"
    footer_style = getSampleStyleSheet()['Normal']
    footer_style.alignment = 1  # 0=Left, 1=Center, 2=Right

    footer = Paragraph(text, footer_style)
    w, h = footer.wrap(doc.width, doc.bottomMargin)
    footer.drawOn(canvas, doc.leftMargin, doc.bottomMargin - h)

    # Change title text color to white for pages after the first one
    if page_num > 1:
        title_style = getSampleStyleSheet()['Title']
        title_style.textColor = colors.white
        main_title.style = title_style


left_margin = 20
right_margin = 20
top_margin = 5
bottom_margin = 20  # Add a bottom margin
page_width = letter[0]  # Get width from letter page size
page_height = letter[1] - top_margin - bottom_margin  # Adjust height for margins

# Create a SimpleDocTemplate with adjusted page size and margins
doc = SimpleDocTemplate(
    str(report_path),
    pagesize=(page_width, page_height),
    leftMargin=left_margin,
    rightMargin=right_margin,
    topMargin=top_margin,
    bottomMargin=bottom_margin
)

# Create a list to hold all elements
elements = []
main_title = Paragraph("Top 50 deliveries by value/volume non-FnO FnO and SME", getSampleStyleSheet()['Title'])
elements.extend([main_title])

# Iterate through your dataframes and add tables to the content frame
for i, df in enumerate(dfs):
    df.name = report_titles[i]
    df_name = df.name
    if i == 2 or i == 3:
        dfs[i] = df[fno_columns_to_select]
    else:
        dfs[i] = df[columns_to_select]

    styles = getSampleStyleSheet()
    h2_style = styles['Title']
    h2_style.fontSize = 9
    title = Paragraph(report_titles[i], style=h2_style)
    table_width = doc.width - doc.leftMargin - doc.rightMargin
    table = create_table_with_header(dfs[i], add_column_names=True, table_width=table_width, df_name=df_name)

    # Add elements for title and table
    records_per_page = 30
    chunks = [dfs[i][j:j + records_per_page] for j in range(0, len(dfs[i]), records_per_page)]
    for chunk in chunks:
        elements.extend([title, create_table_with_header(chunk, add_column_names=True, table_width=table_width, df_name=df_name), PageBreak()])

# Assign the function to be called for each page
doc.build(elements, onFirstPage=add_headers_and_footers, onLaterPages=add_headers_and_footers)


In [335]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, PageBreak
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import Paragraph
from pathlib import Path
from reportlab.pdfbase.pdfmetrics import registerFontFamily

registerFontFamily('Vera', normal='Vera', bold='VeraBd', italic='VeraIt', boldItalic='VeraBI')

columns_to_select = ['Company Name', 'Total_Delivery', 'Price', 'PrevClose', 'Total_Traded_Value', 'NSE_Code']
fno_columns_to_select = ['NSE_Code', 'Total_Delivery', 'Price', 'PrevClose', 'Total_Traded_Value']
dfs = [df_value_50, df_volume_50, fno_value_50, fno_volume_50, sme_value_50]
report_titles = [
    f"Top 50 delivery by value NON-FNO {user_input}",
    f"Top 50 delivery by volume NON-FNO {user_input}",
    f"Top 50 FNO Delivery by value {user_input}",
    f"Top 50 FNO Delivery by volume {user_input}",
    f"Top 50 SME delivery by value {user_input}"
]

#report_prefix_name = f"{user_input}_0_Top 50 deliveries by value volume in non_fno fno and sme.pdf"
#report_path = Path.joinpath(Path(base_folder).parents[0], "Reports", report_prefix_name)
report_prefix_name = f"{user_input}_0_chatGPT_Top 50 deliveries_value_volume_non_fno_fno_sme.pdf"
report_path = os.path.join(reports_path,report_prefix_name)

# Define the style for column headers without grid lines
column_header_style = [
    ('BACKGROUND', (0, 0), (-1, 0), colors.lightgrey),
    ('TEXTCOLOR', (0, 0), (-1, 0), colors.black),
    ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
]


def create_table_with_header(df, df_name, add_column_names=False, table_width=None):
    """create a table with header and data"""
    table_data = [df.columns.tolist()] + df.values.tolist()

    # Add empty rows for spacing after every 5 records
    for i in range(6, len(table_data), 6):
        table_data.insert(i, [''] * len(df.columns))
        
    styles = getSampleStyleSheet()
    h2_style = styles['Normal']
    h2_style.fontSize = 6
    h2_style.fontName = 'Calibri'

    if table_width is None:
        t = Table(table_data, repeatRows=1, style=h2_style)
    else:
        colwdth = [170, 80, 67, 68, 105]
        t = Table(table_data, repeatRows=1, colWidths=colwdth)  # Adjust colWidths for left alignment

    style = [
        ('ALIGN', (0, 0), (0, -1), 'LEFT'),  # Left align first column
        ('ALIGN', (-1, 0), (-1, -1), 'LEFT'),  # Left align last column
        ('ALIGN', (1, 0), (-2, -1), 'RIGHT'),  # Right align other columns
    ]

    # repeatRows=1 to add the first column to all the pages
    t.setStyle(column_header_style)

    if "FNO" in str(df_name) and "NON-FNO" not in str(df_name):
        style = [
            ('ALIGN', (-1, 0), (-1, -1), 'LEFT'),  # Left align first column
            ('ALIGN', (1, 0), (-1, -1), 'RIGHT'),  # Right align other columns (including last)
        ]

    # Set title row color for the first page
    if getattr(create_table_with_header, 'first_page', True):
        style.extend([
            ('BACKGROUND', (0, 0), (-1, 0), colors.lightgrey),  # Background color for the title row
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.black),
        ])
        create_table_with_header.first_page = False

    t.setStyle(style)

    # Remove table borders
    t.setStyle([('LINEBELOW', (0, 0), (-1, 0), 0, colors.white),  # Hide line below header
                ('GRID', (0, 0), (-1, -1), 0, colors.white),
               ('LEADING', (0, 0), (-1, -1), 8)])  # Hide grid lines

    return t

    #t.setStyle([('LEADING', (0, 0), (-1, -1), 12)])  # Adjust leading to set space between lines

    


def add_headers_and_footers(canvas, doc):
    """Add header and footer here"""
    page_num = canvas.getPageNumber()
    text = f"Page {page_num}"
    footer_style = getSampleStyleSheet()['Normal']
    footer_style.alignment = 1  # 0=Left, 1=Center, 2=Right

    footer = Paragraph(text, footer_style)
    w, h = footer.wrap(doc.width, doc.bottomMargin)
    footer.drawOn(canvas, doc.leftMargin, doc.bottomMargin - h)

    # Change title text color to white for pages after the first one
    if page_num > 1:
        title_style = getSampleStyleSheet()['Title']
        title_style.textColor = colors.white
        main_title.style = title_style


left_margin = 20
right_margin = 20
top_margin = 5
bottom_margin = 20  # Add a bottom margin
page_width = letter[0]  # Get width from letter page size
page_height = letter[1] - top_margin - bottom_margin  # Adjust height for margins

# Create a SimpleDocTemplate with adjusted page size and margins
doc = SimpleDocTemplate(
    str(report_path),
    pagesize=(page_width, page_height),
    leftMargin=left_margin,
    rightMargin=right_margin,
    topMargin=top_margin,
    bottomMargin=bottom_margin
)

# Create a list to hold all elements
elements = []
main_title = Paragraph("Top 50 deliveries by value/volume non-FnO FnO and SME", getSampleStyleSheet()['Title'])
elements.extend([main_title])

# Iterate through your dataframes and add tables to the content frame
for i, df in enumerate(dfs):
    df.name = report_titles[i]
    df_name = df.name
    if i == 2 or i == 3:
        dfs[i] = df[fno_columns_to_select]
    else:
        dfs[i] = df[columns_to_select]

    styles = getSampleStyleSheet()
    h2_style = styles['Title']
    h2_style.fontSize = 9
    title = Paragraph(report_titles[i], style=h2_style)
    table_width = doc.width - doc.leftMargin - doc.rightMargin
    table = create_table_with_header(dfs[i], add_column_names=True, table_width=table_width, df_name=df_name)

    # Add elements for title and table
    records_per_page = 30
    chunks = [dfs[i][j:j + records_per_page] for j in range(0, len(dfs[i]), records_per_page)]
    for chunk in chunks:
        elements.extend([title, create_table_with_header(chunk, add_column_names=True, table_width=table_width, df_name=df_name), PageBreak()])

# Assign the function to be called for each page
doc.build(elements, onFirstPage=add_headers_and_footers, onLaterPages=add_headers_and_footers)



In [ ]:
# Adjust row heights and spacing
for i in range(1, len(table_data)):
    t._argW = colwdth  # Set column widths for the entire table
    t._argH[i] = 6  # Set a smaller row height (adjust this value)
    if i % 6 == 0:  # Increase space after every 5 records
        t._argH.append(5)  # Append the desired spacing


In [316]:
##columns selection for printing
columns_to_select = ['Company Name', 'Total_Delivery', 'Price','PrevClose', 'Total_Traded_Value', 'NSE_Code']
col_length = [40,15,12,18,10]
fno_columns_to_select = ['NSE_Code', 'Total_Delivery', 'Price','PrevClose', 'Total_Traded_Value']



print("Top 50 deliveries by value/volume non-FnO FnO and SME ", nse_dely_date)
print("")
print("")
#print(nse_dely_date)
#print()
print("                          Top 50 delivery by value NON FNO ", nse_dely_date)
print("")
print("Company Name                                ", '  Dely_Qty','      Price','   PrevClose','           Value','   NSECode')
print("")
print_formatted_rows(df_value_50,columns_to_select)
print("")
print("")
print("")


print("                        Top 50 delivery by volume NON FNO ", nse_dely_date)
print("")
print("Company Name                                ", '  Dely_Qty','      Price','   PrevClose','           Value','   NSECode')
print("")

print_formatted_rows(df_volume_50,columns_to_select)
print("")
print("")
print("")


#### printing for fno
print("          Top 50 FNO Delivery by value ", nse_dely_date)
print("")
print("NSECode       ",'  Dely_Qty','      Price','   PrevClose','    Value' )
print("")
fno_formatted_rows(fno_value_50,fno_columns_to_select)
print("")
print("")
print("")

print("             Top 50 FNO Delivery by volume ", nse_dely_date)
print("")
print("NSECode       ",'  Dely_Qty','      Price','   PrevClose','   Value' )
print("")

fno_formatted_rows(fno_volume_50,fno_columns_to_select)
print("")
print("")
print("")


print("                          Top 50 delivery by value SME ", nse_dely_date)
print("")
print("Company Name                                ", '  Dely_Qty','      Price','   PrevClose','           Value','   NSECode')
print("")
print_formatted_rows(sme_value_50,columns_to_select)
print("")
print("")
print("")




Top 50 deliveries by value/volume non-FnO FnO and SME  231222


                          Top 50 delivery by value NON FNO  231222

Company Name                                   Dely_Qty       Price    PrevClose            Value    NSECode

IIFL Finance Ltd.                            33,576,956      572.60      595.15        192,261.65    IIFL      
Varun Beverages Ltd.                          3,120,411    1,238.60    1,291.30         38,649.41    VBL       
Indian Railway Finance Corpora               35,082,422       97.75       98.85         34,293.07    IRFC      
Zomato Ltd.                                  24,177,301      128.50      127.35         31,067.83    ZOMATO    
Life Insurance Corporation of                 3,000,680      792.80      764.50         23,789.39    LICI      

                                              1,975,244    1,000.00    1,000.00         19,752.44    LIQUIDBEES
Shyam Metalics And Energy Ltd.                2,938,178      608.40      571.90      

In [317]:
sorted_df = bse_dely_data.sort_values(by='BSE_Delivery', ascending = False)
sorted_df


,BSE_Code,Date,BSE_Delivery
2321,532775,231222,37286463
2236,532636,231222,21655090
2244,532648,231222,14582156
2352,532822,231222,13268906
2877,539277,231222,13111183
...,...,...,...
2728,538556,231222,1
3399,543106,231222,1
4158,954311,231222,1
4160,958406,231222,1


In [318]:
m_segment = final_master_file[final_master_file['Segment'] == "SM"]
m_segment

,Date,Company Name,NSE_Code,BSE_Code,Total_Volume,Total_Delivery,Price,PrevClose,Total_Traded_Value,NSE_Volume,NSE_Delivery,BSE_Volume,BSE_Delivery,Segment


In [319]:
sme_value_50

,Date,Company Name,NSE_Code,BSE_Code,Total_Volume,Total_Delivery,Price,PrevClose,Total_Traded_Value,NSE_Volume,NSE_Delivery,BSE_Volume,BSE_Delivery,Segment
203,231222,S J Logistics,SJLOGISTIC,,786000,"660,000",174.10,165.90,"1,149.06",786000.0,660000.0,0.0,0.0,ST
120,231222,Phantom Digital Effects Ltd.,PHANTOMFX,,271200,"170,400",520.30,479.95,886.59,271200.0,170400.0,0.0,0.0,SM
301,231222,Insolation Energy Ltd.,543620,543620,120500,"120,500",696.20,677.50,838.92,0.0,0.0,120500.0,120500.0,MT
195,231222,Rocking Deals Circular Economy,ROCKINGDCE,,382000,"229,000",331.90,276.60,760.05,382000.0,229000.0,0.0,0.0,SM
20,231222,Basilic Fly Studio,BASILIC,,327600,"242,400",301.95,284.85,731.93,327600.0,242400.0,0.0,0.0,SM
309,231222,Droneacharya Aerial Innovation,543713,543713,625000,"388,000",185.75,196.25,720.71,0.0,0.0,625000.0,388000.0,M
200,231222,Accent Microcell Limited,ACCENTMIC,,251000,"228,000",288.55,279.20,657.89,251000.0,228000.0,0.0,0.0,ST
184,231222,Vinyas Innovative Technologies,VINYAS,,102400,"64,800",735.55,639.95,476.64,102400.0,64800.0,0.0,0.0,SM
114,231222,Oriana Power Limited,ORIANA,,136500,"84,000",501.80,506.65,421.51,136500.0,84000.0,0.0,0.0,SM
142,231222,Sahana System Ltd.,SAHANA,,96000,"65,000",617.10,589.45,401.12,96000.0,65000.0,0.0,0.0,SM
